In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
os.listdir('../input/cwt-image-dataset')

['target0',
 '__results__.html',
 '__pycache__',
 'target1',
 '__notebook__.ipynb',
 '__results___files',
 '__output__.json',
 'tf_cwt.py',
 'custom.css']

In [3]:
train_datagen = ImageDataGenerator(zoom_range=0.1,validation_split=0.2)

In [4]:
BATCH_SIZE=50
train_generator = train_datagen.flow_from_directory('../input/cwt-image-dataset',
    target_size=(90, 224),
    batch_size=BATCH_SIZE,
    class_mode='binary',subset='training',shuffle=True, classes=["target0", "target1"])
validation_generator = train_datagen.flow_from_directory('../input/cwt-image-dataset',
    target_size=(90, 224),
    batch_size=BATCH_SIZE,
    class_mode='binary',subset='validation',shuffle=True, classes=["target0", "target1"])

Found 96000 images belonging to 2 classes.
Found 24000 images belonging to 2 classes.


In [5]:
effnet=EfficientNetB6(weights = 'imagenet', input_shape = (90, 224, 3), include_top = False)
for l in effnet.layers:
    l.trainable = True

165240832/165234480 [==============================] - 1s 0us/step


## Here is Adam optimizer

In [6]:
new_model=Sequential()
new_model.add(effnet)
new_model.add(Flatten())
new_model.add(Dense(256, activation='relu'))
new_model.add(Dropout(0.2))
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.2))
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.2))
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.2))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(optimizer = Adam(lr=1e-3),loss='binary_crossentropy',metrics=['AUC'])

In [7]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb6 (Functional)  (None, 3, 7, 2304)        40960143  
_________________________________________________________________
flatten (Flatten)            (None, 48384)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               12386560  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1

In [10]:
epochs=15
es = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=4)
lr_reducer = ReduceLROnPlateau(monitor='val_auc',factor=np.sqrt(0.1), cooldown=0,\
                               patience=3, min_lr=0.5e-8,mode='max')
# This file path where the best weights will be saved
checkpoint_filepath = './effB6_CWT_adam_weights20.best.hdf5'
# It saves the weights which shows the highest validation accuracy
modelchck=ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,verbose=1,
                          monitor='val_auc',mode='max',save_best_only=True)

new_model.fit(train_generator, steps_per_epoch=train_generator.n // BATCH_SIZE,\
              epochs=epochs, validation_data=validation_generator,\
              validation_steps=validation_generator.n // BATCH_SIZE,\
              callbacks=[lr_reducer,modelchck,es])

Epoch 1/15
1920/1920 [==============================] - 1318s 671ms/step - loss: 0.5771 - auc: 0.7377 - val_loss: 0.5068 - val_auc: 0.8408

Epoch 00001: val_auc improved from -inf to 0.84083, saving model to ./effB6_CWT_adam_weights20.best.hdf5
Epoch 2/15
1920/1920 [==============================] - 1214s 632ms/step - loss: 0.4550 - auc: 0.8429 - val_loss: 1.1294 - val_auc: 0.8077

Epoch 00002: val_auc did not improve from 0.84083
Epoch 3/15
1920/1920 [==============================] - 1216s 633ms/step - loss: 0.4378 - auc: 0.8535 - val_loss: 0.4431 - val_auc: 0.8531

Epoch 00003: val_auc improved from 0.84083 to 0.85312, saving model to ./effB6_CWT_adam_weights20.best.hdf5
Epoch 4/15
1920/1920 [==============================] - 1220s 636ms/step - loss: 0.4287 - auc: 0.8597 - val_loss: 0.4545 - val_auc: 0.8541

Epoch 00004: val_auc improved from 0.85312 to 0.85409, saving model to ./effB6_CWT_adam_weights20.best.hdf5
Epoch 5/15
1920/1920 [==============================] - 1220s 636ms/s

## New effnetb6 boarder and deeper with Adam

In [6]:
new_model=Sequential()
new_model.add(effnet)
new_model.add(GlobalAveragePooling2D())
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.25))
new_model.add(Dense(32, activation='relu'))
new_model.add(Dropout(0.25))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(optimizer = Adam(lr=1e-3),loss='binary_crossentropy',metrics=['AUC'])

In [7]:
epochs=15
es = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=5)
lr_reducer = ReduceLROnPlateau(monitor='val_auc',factor=np.sqrt(0.1), cooldown=0,\
                               patience=3, min_lr=0.5e-8,mode='max')
# This file path where the best weights will be saved
checkpoint_filepath = './effB6_CWT_adam_weights20.best.hdf5'
# It saves the weights which shows the highest validation accuracy
modelchck=ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,verbose=1,
                          monitor='val_auc',mode='max',save_best_only=True)

new_model.fit(train_generator, steps_per_epoch=train_generator.n // BATCH_SIZE,\
              epochs=epochs, validation_data=validation_generator,\
              validation_steps=validation_generator.n // BATCH_SIZE,\
              callbacks=[lr_reducer,modelchck,es])

Epoch 1/15
1920/1920 [==============================] - 1639s 837ms/step - loss: 0.5352 - auc: 0.7707 - val_loss: 0.4733 - val_auc: 0.8389

Epoch 00001: val_auc improved from -inf to 0.83892, saving model to ./effB6_CWT_adam_weights20.best.hdf5
Epoch 2/15
1920/1920 [==============================] - 1327s 691ms/step - loss: 0.4504 - auc: 0.8448 - val_loss: 0.4548 - val_auc: 0.8501

Epoch 00002: val_auc improved from 0.83892 to 0.85008, saving model to ./effB6_CWT_adam_weights20.best.hdf5
Epoch 3/15
1920/1920 [==============================] - 1331s 693ms/step - loss: 0.4437 - auc: 0.8505 - val_loss: 0.4473 - val_auc: 0.8501

Epoch 00003: val_auc did not improve from 0.85008
Epoch 4/15
1920/1920 [==============================] - 1329s 692ms/step - loss: 0.4350 - auc: 0.8542 - val_loss: 0.4412 - val_auc: 0.8570

Epoch 00004: val_auc improved from 0.85008 to 0.85704, saving model to ./effB6_CWT_adam_weights20.best.hdf5
Epoch 5/15
1920/1920 [==============================] - 1332s 694ms/s

## Build the test set

In [8]:
checkpoint_filepath = './effB6_CWT_adam_weights20.best.hdf5'
new_model.load_weights(checkpoint_filepath)

In [9]:
test_datagen = ImageDataGenerator()
BATCH_SIZE=100
test_generator = test_datagen.flow_from_directory('../input/cwt-test-data',\
                                                  target_size=(90, 224),\
                                                  batch_size=BATCH_SIZE,\
                                                  class_mode=None,shuffle=False,\
                                                  classes=["test_data_cwt"])

Found 226000 images belonging to 1 classes.


In [10]:
y_pred=new_model.predict(test_generator)

In [11]:
filenames=test_generator.filenames
filenames[0]

'test_data_cwt/id_00005bced6.png'

In [12]:
edit_filenames=[]
for i in filenames:
    edit_filenames.append(i[17:-4])

In [13]:
output = pd.DataFrame({'Id': edit_filenames, 'target': y_pred.flatten()})
output.head()

,Id,target
0,00005bced6,1.000000
1,0000806717,0.985269
2,0000ef4fe1,0.625282
3,00020de251,0.601625
4,00024887b5,0.213722


In [14]:
output.to_csv('./effnetb6_mod_CWT_small_batch.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
